In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta

# Importa as duas funções que criamos no arquivo data_loader.py
from src_mvp.data_loader import get_historical_ohlcv, get_news_data

print("Funções importadas com sucesso!")

Funções importadas com sucesso!


In [2]:
# --- Parâmetros do Teste de Preço ---
ATIVO = 'BTC/USDT'
TIMEFRAME = '1d'       # 1 dia (para um backtest mais longo)
DATA_INICIO_PRECO = '2018-01-01' # Pegar um bom histórico
PASTA_SAIDA = 'data/raw'

print(f"--- Iniciando Teste 1: Coleta de Preços para {ATIVO} ---")

# --- Executa a função ---
df_precos = get_historical_ohlcv(ATIVO, TIMEFRAME, DATA_INICIO_PRECO, PASTA_SAIDA)

# --- Verifica o resultado ---
if df_precos is not None:
    print(f"\nSucesso! {len(df_precos)} registros baixados.")
    print("Últimos 5 registros:")
    print(df_precos.tail())
else:
    print("\nFalha na coleta de preços.")

print("--- Teste 1 Concluído ---")

--- Iniciando Teste 1: Coleta de Preços para BTC/USDT ---
Buscando dados para BTC/USDT no timeframe 1d desde 2018-01-01...
  > Recebidos 1000 velas...
  > Recebidos 1000 velas...
  > Recebidos 862 velas...

Busca concluída. Total de 2862 registros baixados.
Dados salvos com sucesso em: data/raw\ohlcv_BTC_USDT_1d.csv

Sucesso! 2862 registros baixados.
Últimos 5 registros:
                 open       high        low      close       volume
datetime                                                           
2025-10-28  114107.65  116086.00  112211.00  112898.45  15523.42257
2025-10-29  112898.44  113643.73  109200.00  110021.29  21079.71376
2025-10-30  110021.30  111592.00  106304.34  108322.88  25988.82838
2025-10-31  108322.87  111190.00  108275.28  109608.01  21518.20439
2025-11-01  109608.01  109877.00  109394.81  109732.33    381.80286
--- Teste 1 Concluído ---


In [3]:
# --- Parâmetros do Teste de Notícias ---
# Lembre-se: O plano gratuito da NewsAPI só busca os últimos 30 dias.
QUERY_NOTICIAS = 'Bitcoin OR Crypto OR BTC'
DATA_FIM_NOTICIA = datetime.now()
DATA_INICIO_NOTICIA = DATA_FIM_NOTICIA - timedelta(days=28) # Buscando 28 dias atrás

# Formata as datas para a API
data_fim_str = DATA_FIM_NOTICIA.strftime('%Y-%m-%d')
data_inicio_str = DATA_INICIO_NOTICIA.strftime('%Y-%m-%d')

print(f"\n--- Iniciando Teste 2: Coleta de Notícias ---")
print(f"Buscando por '{QUERY_NOTICIAS}' de {data_inicio_str} até {data_fim_str}")

# --- Executa a função ---
dados_noticias = get_news_data(QUERY_NOTICIAS, data_inicio_str, data_fim_str, PASTA_SAIDA)

# --- Verifica o resultado ---
if dados_noticias:
    total_noticias = dados_noticias.get('totalResults', 0)
    print(f"\nSucesso! Total de {total_noticias} notícias encontradas.")
    if total_noticias > 0:
        print(f"Exemplo (primeira notícia): {dados_noticias['articles'][0]['title']}")
else:
    print("\nFalha na coleta de notícias. Verifique sua chave de API no .env")

print("--- Teste 2 Concluído ---")


--- Iniciando Teste 2: Coleta de Notícias ---
Buscando por 'Bitcoin OR Crypto OR BTC' de 2025-10-03 até 2025-10-31
Buscando notícias para 'Bitcoin OR Crypto OR BTC' de 2025-10-03 até 2025-10-31...
Total de 16110 artigos encontrados.
Notícias salvas com sucesso em: data/raw\noticias_bitcoin_or_crypto_or_btc_2025-10-03_a_2025-10-31.json

Sucesso! Total de 16110 notícias encontradas.
Exemplo (primeira notícia): How the Acrisure Arena Became an Unlikely Desert Oasis for Music Fans From Around California… and a Favorite Kickoff Point for Some Major Tours
--- Teste 2 Concluído ---


In [4]:
print("--- Verificando arquivos na pasta 'data/raw' ---")

pasta_raw = 'data/raw'
arquivos_em_raw = os.listdir(pasta_raw)

if not arquivos_em_raw:
    print("Pasta 'data/raw' está vazia.")
else:
    for arquivo in arquivos_em_raw:
        if arquivo.endswith('.csv') or arquivo.endswith('.json'):
            print(f"Arquivo encontrado: {arquivo}")

print("-----------------------------------------------")

--- Verificando arquivos na pasta 'data/raw' ---
Arquivo encontrado: noticias_bitcoin_or_crypto_or_btc_2025-10-03_a_2025-10-31.json
Arquivo encontrado: ohlcv_BTC_USDT_1d.csv
-----------------------------------------------


In [5]:
import os
from src_mvp.feature_builder import analisar_sentimento_noticias

print("--- Iniciando Teste 3: Análise de Sentimento (Fase 3) ---")

# --- Parâmetros ---
PASTA_RAW = 'data/raw'
PASTA_PROCESSED = 'data/processed'

# !!! ATENÇÃO !!!
# Coloque o nome EXATO do arquivo .json que você criou na Fase 2
# (Você pode ver o nome na saída da Célula 4)
NOME_ARQUIVO_NOTICIAS = 'noticias_bitcoin_or_crypto_or_btc_2025-10-03_a_2025-10-31.json' # <--- MUDE AQUI

caminho_json_entrada = os.path.join(PASTA_RAW, NOME_ARQUIVO_NOTICIAS)

# --- Executa a função ---
# (Isso pode levar alguns minutos, pois faz 1 chamada de API a cada 2 segundos)
df_sentimento = analisar_sentimento_noticias(caminho_json_entrada, PASTA_PROCESSED)

# --- Verifica o resultado ---
if df_sentimento is not None:
    print("\nSucesso! Análise de sentimento concluída.")
    print("Amostra dos resultados:")
    print(df_sentimento[['sentiment_score', 'reasoning']].head())
    
    print("\nVerificando arquivo salvo...")
    arquivos_em_processed = os.listdir(PASTA_PROCESSED)
    print(f"Arquivos em 'data/processed': {arquivos_em_processed}")
else:
    print("\nFalha na análise de sentimento.")

print("--- Teste 3 Concluído ---")

--- Iniciando Teste 3: Análise de Sentimento (Fase 3) ---
API do Google Gemini configurada com sucesso.
Carregados 99 artigos de 'data/raw\noticias_bitcoin_or_crypto_or_btc_2025-10-03_a_2025-10-31.json'
  > Sucesso (Score: 0.00): How the Acrisure Arena Became an Unlikely Desert O...
  > Sucesso (Score: -0.75): Elizabeth Warren Lashes Out At Trump's Move To Int...
  > Sucesso (Score: -0.85): Fed Crushes Caitlin Long’s Crypto Bank’s 5-Year Bi...
  > Sucesso (Score: 0.75): Does Bitcoin Power Law model still work in 2025 af...
  > Sucesso (Score: -0.70): Bitcoin Point Of Control Sits At $117K – Key Battl...
  > Sucesso (Score: 0.70): Stablecoin Issuer Tether Releases Attestation Repo...
  > Sucesso (Score: 0.00): Pundit Breaks Down The XRP Ledger: What To Know Ab...
  > Sucesso (Score: 0.85): The first AI launchpad on Sui: Empowering retail i...
  > Sucesso (Score: 0.40): From terrorism to ‘Wrench Attacks’: Inside T3’s $3...
  > Sucesso (Score: 1.00): Global Money Supply Is Increasing, Sup

In [7]:
import os
# --- Força o Jupyter a reler o arquivo .py do disco ---
import importlib
import src_mvp.feature_builder
importlib.reload(src_mvp.feature_builder)
# --- Fim do código de recarregamento ---

from src_mvp.feature_builder import calcular_indicadores_tecnicos

print("--- Iniciando Teste 4: Cálculo de Indicadores Técnicos (Fase 3) ---")

# --- Parâmetros ---
PASTA_RAW = 'data/raw'
PASTA_PROCESSED = 'data/processed'

# !!! ATENÇÃO !!!
# Coloque o nome EXATO do arquivo .csv de PREÇOS que você criou na Fase 2
# (Você pode ver o nome na saída da Célula 4)
NOME_ARQUIVO_PRECOS = 'ohlcv_BTC_USDT_1d.csv' # <--- MUDE AQUI SE NECESSÁRIO

caminho_csv_entrada = os.path.join(PASTA_RAW, NOME_ARQUIVO_PRECOS)

# --- Executa a função ---
df_precos_features = calcular_indicadores_tecnicos(caminho_csv_entrada, PASTA_PROCESSED)

# --- Verifica o resultado ---
if df_precos_features is not None:
    print("\nSucesso! Indicadores técnicos calculados.")
    print("Visualizando as novas colunas (features):")
    # Mostra as colunas novas
    print(df_precos_features[['close', 'ma_curta', 'ma_longa', 'rsi', 'momentum_5d']].tail())
    
    print("\nVerificando arquivo salvo...")
    arquivos_em_processed = os.listdir(PASTA_PROCESSED)
    print(f"Arquivos em 'data/processed': {arquivos_em_processed}")
else:
    print("\nFalha no cálculo dos indicadores.")

print("--- Teste 4 Concluído ---")

--- Iniciando Teste 4: Cálculo de Indicadores Técnicos (Fase 3) ---
Carregados 2862 registros de preços de 'data/raw\ohlcv_BTC_USDT_1d.csv'
Indicadores calculados. 2813 registros válidos restantes.
Features de preço salvas com sucesso em: data/processed\features_precos_BTC_USDT_1d.csv

Sucesso! Indicadores técnicos calculados.
Visualizando as novas colunas (features):
                close    ma_curta     ma_longa        rsi  momentum_5d
datetime                                                              
2025-10-28  112898.45  110933.482  114276.8154  49.709403     0.025621
2025-10-29  110021.29  111071.333  114246.3134  48.381823    -0.008861
2025-10-30  108322.88  110850.412  114133.5710  50.291529    -0.029767
2025-10-31  109608.01  110981.446  114016.0774  57.359884    -0.043221
2025-11-01  109732.33  111197.935  113890.1356  56.079621    -0.038344

Verificando arquivo salvo...
Arquivos em 'data/processed': ['.gitkeep', 'features_precos_BTC_USDT_1d.csv', 'sentimento_noticias.csv

In [8]:
import os
# --- Força o Jupyter a reler o arquivo .py do disco ---
import importlib
import src_mvp.feature_builder
importlib.reload(src_mvp.feature_builder)
# --- Fim do código de recarregamento ---

from src_mvp.feature_builder import criar_dataset_final

print("--- Iniciando Teste 5: Criação do Dataset Final (Fase 3) ---")

# --- Parâmetros ---
PASTA_PROCESSED = 'data/processed'

# Use os nomes dos arquivos que você criou nos testes 3 e 4
# (Eles devem estar na pasta 'data/processed')
NOME_ARQUIVO_FEATURES_PRECOS = 'features_precos_BTC_USDT_1d.csv' # <--- Verifique este nome
NOME_ARQUIVO_SENTIMENTO = 'sentimento_noticias.csv'

caminho_precos = os.path.join(PASTA_PROCESSED, NOME_ARQUIVO_FEATURES_PRECOS)
caminho_sentimento = os.path.join(PASTA_PROCESSED, NOME_ARQUIVO_SENTIMENTO)


# --- Executa a função ---
df_dataset_final = criar_dataset_final(caminho_precos, caminho_sentimento, PASTA_PROCESSED)

# --- Verifica o resultado ---
if df_dataset_final is not None:
    print("\nSucesso! Dataset final criado.")
    print("Visualizando as colunas finais (incluindo 'target' e 'sentimento'):")
    
    # Mostra colunas de preço, features, sentimento e o target
    print(df_dataset_final[['close', 'rsi', 'sentimento_medio', 'target_preco_subiu']].tail())
    
    print("\nVerificando arquivo salvo...")
    arquivos_em_processed = os.listdir(PASTA_PROCESSED)
    print(f"Arquivos em 'data/processed': {arquivos_em_processed}")
else:
    print("\nFalha ao criar o dataset final.")

print("--- Teste 5 Concluído ---")

--- Iniciando Teste 5: Criação do Dataset Final (Fase 3) ---
Arquivos de features de preço e sentimento carregados e padronizados para UTC.
Sentimento agregado por dia (média diária).
Dataset final unido e coluna 'target' criada.
Dataset final salvo com sucesso em: data/processed\dataset_final_treinamento.parquet

Sucesso! Dataset final criado.
Visualizando as colunas finais (incluindo 'target' e 'sentimento'):
                               close        rsi  sentimento_medio  \
datetime                                                            
2025-10-27 00:00:00+00:00  114107.65  47.723302             0.000   
2025-10-28 00:00:00+00:00  112898.45  49.709403             0.000   
2025-10-29 00:00:00+00:00  110021.29  48.381823             0.000   
2025-10-30 00:00:00+00:00  108322.88  50.291529             0.000   
2025-10-31 00:00:00+00:00  109608.01  57.359884            -0.036   

                           target_preco_subiu  
datetime                                       
2025-